In [1]:
import pandas as pd 
import numpy as np
import os
import math
import scipy.io as io
from tqdm.notebook import tqdm

import matplotlib.pyplot as plt
%matplotlib qt

HOME = os.environ['HOME']

# Cleaning Data

In [2]:
lstat = pd.read_pickle(f'{HOME}/webots_code/data/final/test.pkl')
# mstat = pd.read_pickle(f'{HOME}/webots_code/data/traffic_test/med/stats_pd.pkl')
# hstat = pd.read_pickle(f'{HOME}/webots_code/data/traffic_test/high/stats_pd.pkl')

In [4]:
# print(len(lstat),len(mstat),len(hstat))

In [5]:
def check_df(traf:str):
    
    '''
    traf : 'low','med','high'
    '''
    
    if(traf=='low'):
        dpath = f'{HOME}/webots_code/data/traffic_test/low/labels'
        label = os.listdir(dpath)
        nf = list()
        for i in lstat.index.values:
            t = lstat.at[i,'Time']
#             print(lstat.at[i,'Name']+f'{t:.1f}'+'.mat',label[0])
            if not(lstat.at[i,'Name']+f'{t:.1f}'+'.mat' in label):
                nf.append(i)
        return nf
    elif(traf=='med'):
        dpath = f'{HOME}/webots_code/data/traffic_test/med/labels'
        label = os.listdir(dpath)
        nf = list()
        for i in mstat.index.values:
            t = mstat.at[i,'Time']
            if not(mstat.at[i,'Name']+f'{t:.1f}'+'.mat' in label):
                nf.append(i)
        return nf
    else:
        dpath = f'{HOME}/webots_code/data/traffic_test/high/labels'
        label = os.listdir(dpath)
        nf = list()
        for i in hstat.index.values:
            t = hstat.at[i,'Time']
            if not(hstat.at[i,'Name']+f'{t:.1f}'+'.mat' in label):
                nf.append(i)
        return nf

In [5]:
# lstat = lstat.drop(index=check_df('low'))
# mstat = mstat.drop(index=check_df('med'))
# hstat = hstat.drop(index=check_df('high'))

In [6]:
# print(len(lstat),len(mstat),len(hstat))
# lstat.to_pickle(f'{HOME}/webots_code/data/traffic_test/low/stats_pd.pkl')
# mstat.to_pickle(f'{HOME}/webots_code/data/traffic_test/med/stats_pd.pkl')
# hstat.to_pickle(f'{HOME}/webots_code/data/traffic_test/high/stats_pd.pkl')

1207 1222 1496


In [59]:
# def find_cnt(lw:int,hg:int):
#     print(sum(lw<=lstat['veh_range']< hg) + sum(lw<=mstat['veh_range']<hg) + sum(lw<=hstat['veh_range']<hg))

In [60]:
# find_cnt(0,5)

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

# Loading model

In [3]:
import torch 
from torch import nn
from torch.nn import functional as F

import pytorch_lightning as pl

In [4]:
class incp(nn.Module):
    def __init__(self,drp:float=0.3,
                 drp_fc:float=0.2):
        super().__init__()
        
        self.drop_CNN = nn.Dropout(drp)
        self.drop_fc = nn.Dropout(drp_fc)
        self.mpool = nn.MaxPool2d((2,2))
        self.channels = 5
        self.fchannel = 3
        self.conv1 = self.create_conv(10,self.channels,13)
        self.conv2 = self.create_conv(self.channels,self.channels,13)
        self.conv3 = self.create_conv(self.channels,self.channels,7)
        self.conv4 = self.create_conv(self.channels,self.channels,7)

        # 1st Inception block
        self.in1 = nn.Sequential(
            self.create_conv(self.channels,self.fchannel,1),
            self.create_conv(self.fchannel,self.fchannel,3),
            self.create_conv(self.fchannel,self.fchannel,3)
        )
        self.in2 = nn.Sequential(
            self.create_conv(self.channels,self.fchannel,1),
            self.create_conv(self.fchannel,self.fchannel,3)
        )
        self.in3 = nn.Sequential(
            nn.MaxPool2d((3,3)),
            self.create_conv(self.channels,self.fchannel,1)
        )
        self.in4 = self.create_conv(self.channels,self.fchannel,1)

        # Second inception block
        self.in5 = nn.Sequential(
            self.create_conv(self.fchannel*4,self.fchannel,1),
            self.create_conv(self.fchannel,self.fchannel,3),
            self.create_conv(self.fchannel,self.fchannel,3)
        )
        self.in6 = nn.Sequential(
            self.create_conv(self.fchannel*4,self.fchannel,1),
            self.create_conv(self.fchannel,self.fchannel,3)
        )
        self.in7 = nn.Sequential(
            nn.MaxPool2d((3,3)),
            self.create_conv(self.fchannel*4,self.fchannel,1)
        )
        self.in8 = self.create_conv(self.fchannel*4,self.fchannel,1)

        self.conv5 = self.create_conv(self.fchannel*4,self.fchannel,7)
        self.conv6 = self.create_conv(self.fchannel,self.fchannel,7)

        self.conv7 = self.create_conv(self.fchannel,self.fchannel,3)
        self.conv8 = self.create_conv(self.fchannel,self.fchannel,3)
        
        self.flatten = nn.Flatten()

        # self.linear = nn.Sequential(
        #     nn.Linear(2352,256),
        #     nn.PReLU(1),
        #     nn.BatchNorm1d(256),
        #     self.drop_fc,
        #     nn.Linear(256,64),
        #     nn.PReLU(1),
        #     nn.BatchNorm1d(64),
        #     self.drop_fc,
        #     nn.Linear(64,16),
        #     nn.PReLU(1),
        #     nn.BatchNorm1d(16),
        #     self.drop_fc,
        #     nn.Linear(16,3)
        # )

        self.linear = nn.Sequential(
            nn.Linear(588,16),
            nn.PReLU(1),
            nn.BatchNorm1d(16),
            nn.Linear(16,3)
        )

    
    def forward(self,X):
        X  = self.conv1(X)
        X  = self.conv2(X)
        X = self.mpool(X)

        X = self.conv3(X)
        X = self.conv4(X)
        X = self.mpool(X)

        X = self.drop_CNN(X)

        a = self.in1(X)
        a = F.pad(a,[2,2,2,2])
        b = self.in2(X)
        b = F.pad(b,[1,1,1,1])
        c = self.in3(X)
        c = F.pad(c,[16,16,16,16])
        d = self.in4(X)

        X = torch.cat((a,b,c,d),1)

        X = self.drop_CNN(X)

        a = self.in5(X)
        a = F.pad(a,[2,2,2,2])
        b = self.in6(X)
        b = F.pad(b,[1,1,1,1])
        c = self.in7(X)
        c = F.pad(c,[16,16,16,16])
        d = self.in8(X)

        X = torch.cat((a,b,c,d),1)
        X = self.drop_CNN(X)

        X = self.conv5(X)
        X = self.conv6(X)

        X = self.mpool(X)

        X = self.drop_CNN(X)

        X = self.conv7(X)
        X = self.conv8(X)
        # print(X.shape)

        X = self.flatten(X)
        out = self.linear(X)

        return out
    
    def create_conv(self,in_layers:int,out_layers:int,kernel,stride:int=1,
                    padding:int=0) -> nn.Module :

        return nn.Sequential(
            nn.Conv2d(in_layers,out_layers,kernel,stride,padding),
            nn.PReLU(out_layers),
            nn.BatchNorm2d(out_layers)
        )

In [5]:
class lidar_trainer(pl.LightningModule):
    def __init__(self,use_model:str='incp',
                 drop_prob:float = 0.3,
                 drop_prob_fc:float = 0.2,
                 learning_rate:float = 1e-3,
                 weight_decay:float = 0.0):
        
        '''
        use_model = 'NU','imp'
        '''

        self.drp = drop_prob
        self.drp_fc = drop_prob_fc

        super().__init__()
        if use_model == 'NU':
            self.model = NU(drp=drop_prob,drp_fc = drop_prob_fc)
        elif use_model == 'imp':
            self.model = imp_lidar(drp=drop_prob,drp_fc = drop_prob_fc)
        elif use_model == 'm1':
            self.model = m1(drp=drop_prob,drp_fc = drop_prob_fc)
        elif use_model == 'incp':
            self.model = incp(drp=drop_prob,drp_fc = drop_prob_fc)

        self.celoss = nn.CrossEntropyLoss()
        self.lr = learning_rate
        self.wd = weight_decay

        self.train_acc = 0
        self.val_acc = 0
        self.test_acc = 0

        self.example_input_array = torch.rand(1,10,240,240) # For logging graph
    
    def forward(self,lidar):
        out = self.model(lidar)
        return out
    
    def training_step(self,batch,batch_idx):
        lidar = batch['lidar'].float()
        label = batch['label'].long()
        
        yhat = self(lidar)
        
        loss = self.celoss(yhat,label)
        
        self.log('Train Loss',loss)
        
        return {'loss':loss,'pred':yhat.cpu().detach(),'label':label.cpu().detach()}
    
    def configure_optimizers(self):
        opt = torch.optim.Adam(self.parameters(),self.lr,
                               weight_decay=self.wd)
        return opt
    
    def training_epoch_end(self,train_out):        
        len_out = len(train_out)
        y_pred = torch.Tensor(len_out*BATCH_SIZE,num_BS)
        y_true = torch.Tensor(len_out*BATCH_SIZE)

        for i in range(0,len_out):
            y_pred[i*BATCH_SIZE:(i+1)*BATCH_SIZE,:] = train_out[i]['pred'] 
            y_true[i*BATCH_SIZE:(i+1)*BATCH_SIZE] = train_out[i]['label']

        top1 = top_k_acc(y_true,y_pred,k=1)

        # Calculating Avg loss
        avg_loss = torch.stack([x['loss'] for x in train_out]).mean()
        
        self.logger.experiment.add_scalar('Loss-Train per epoch',avg_loss,self.current_epoch)
        self.logger.experiment.add_scalar('Train Accuracy',top1,self.current_epoch)

        self.train_acc = top1
        
        print(f'Train accuracies is {top1}')

    def validation_step(self,batch,batch_idx):
        lidar = batch['lidar'].float()
        label = batch['label'].long()
        
        yhat = self.forward(lidar)
        
        return [yhat.cpu().detach(),label.cpu().detach()]
     
    def validation_epoch_end(self,val_out):
        len_out = len(val_out)
        y_pred = torch.Tensor(len_out*BATCH_SIZE,num_BS)
        y_true = torch.Tensor(len_out*BATCH_SIZE)

        for i in range(0,len_out):
            y_pred[i*BATCH_SIZE:(i+1)*BATCH_SIZE,:] = val_out[i][0] 
            y_true[i*BATCH_SIZE:(i+1)*BATCH_SIZE] = val_out[i][1] 

        top1 = top_k_acc(y_true,y_pred,k=1)
        self.val_acc = top1

        self.logger.experiment.add_scalar('Validation Accuracy',top1,self.current_epoch)

        print(f'Validation accuracy is {top1}')
    
    def test_step(self,batch,batch_idx):
        lidar = batch['lidar'].float()
        label = batch['label'].long()
        
        yhat = self.forward(lidar)
        
        return [yhat.cpu().detach(),label.cpu().detach()]
     
    def test_epoch_end(self,val_out):
        len_out = len(val_out)
        y_pred = torch.Tensor(len_out*BATCH_SIZE,num_BS)
        y_true = torch.Tensor(len_out*BATCH_SIZE)

        for i in range(0,len_out):
            y_pred[i*BATCH_SIZE:(i+1)*BATCH_SIZE,:] = val_out[i][0] 
            y_true[i*BATCH_SIZE:(i+1)*BATCH_SIZE] = val_out[i][1] 

        top1 = top_k_acc(y_true,y_pred,k=1)
        self.test_acc = top1

        self.logger.experiment.add_hparams(
            {
                'LR' : self.lr,
                'weight_decay' : self.wd,
                'drop_prob' : self.drp,
                'drop_fc' : self.drp_fc,
                'overall params' : sum(p.numel() for p in self.model.parameters())
            },
            {
                'hparam/test_acc' : self.test_acc,
                'hparam/train_acc' : self.train_acc,
                'hparam/val_acc' : self.val_acc
            }
        )
        print(f'Test accuracy is {top1}')

In [6]:
lidar_model = lidar_trainer(use_model='incp',drop_prob=0.3,drop_prob_fc=0.2,weight_decay=1e-4,learning_rate=3.63e-4)

In [7]:
lidar_model = lidar_trainer.load_from_checkpoint('/home/mohit/webots_code/comms_lidar_ML/BS_Selection/tb_logs_new2/inception/version_0/checkpoints/epoch=19-step=3759.ckpt')

In [8]:
model = lidar_model.model

# Evaluting the model

In [9]:
lstat = lstat.reset_index()
# mstat = mstat.reset_index()
# hstat = hstat.reset_index()

In [10]:
lstat.at[0,'Lidar'][:-3]

'SUMO vehicle 301849.8.'

In [22]:
model.eval()
dpath = f'{HOME}/webots_code/data/final'
acc = list()
for i in tqdm(lstat.index.values):
    name = lstat.at[i,'Lidar']
    lidar_data = torch.tensor(np.load(dpath+f'/lidar_compressed/{name}')['lidar'].reshape(1,10,240,240).astype('float32'))   
    label = io.loadmat(dpath+f'/labels/{name[:-3]}mat')['ss']
    label = np.argmax(label.astype('float64'))
    
    pred = model(lidar_data)
    
#     print(label,torch.argmax(pred))
    if (label == torch.argmax(pred)):
        acc.append(1)
    else:
        acc.append(0)

  0%|          | 0/1342 [00:00<?, ?it/s]

In [23]:
sum(acc)/len(acc)

0.6035767511177347

In [24]:
lstat['Pred'] = acc

In [17]:
model.eval()
dpath = f'{HOME}/webots_code/data/traffic_test/med'
acc = list()
for i in tqdm(mstat.index.values):
    name = mstat.at[i,'Lidar']
    lidar_data = torch.tensor(np.load(dpath+f'/lidar_compressed/{name}')['lidar'].reshape(1,10,240,240).astype('float32'))   
    label = io.loadmat(dpath+f'/labels/{name[:-3]}mat')['ss']
    label = np.argmax(label.astype('float64'))
    
    pred = model(lidar_data)
    
#     print(label,torch.argmax(pred))
    if (label == torch.argmax(pred)):
        acc.append(1)
    else:
        acc.append(0)
print(sum(acc)/len(acc))

  0%|          | 0/1222 [00:00<?, ?it/s]

0.6399345335515548


In [20]:
mstat['Pred'] = acc

In [21]:
model.eval()
dpath = f'{HOME}/webots_code/data/traffic_test/high'
acc = list()
for i in tqdm(hstat.index.values):
    name = hstat.at[i,'Lidar']
    lidar_data = torch.tensor(np.load(dpath+f'/lidar_compressed/{name}')['lidar'].reshape(1,10,240,240).astype('float32'))   
    label = io.loadmat(dpath+f'/labels/{name[:-3]}mat')['ss']
    label = np.argmax(label.astype('float64'))
    
    pred = model(lidar_data)
    
#     print(label,torch.argmax(pred))
    if (label == torch.argmax(pred)):
        acc.append(1)
    else:
        acc.append(0)
print(sum(acc)/len(acc))
hstat['Pred'] = acc

  0%|          | 0/1496 [00:00<?, ?it/s]

0.6082887700534759


In [22]:
# print(len(lstat),len(mstat),len(hstat))
# lstat.to_pickle(f'{HOME}/webots_code/data/traffic_test/low/stats_pd.pkl')
# mstat.to_pickle(f'{HOME}/webots_code/data/traffic_test/med/stats_pd.pkl')
# hstat.to_pickle(f'{HOME}/webots_code/data/traffic_test/high/stats_pd.pkl')

1207 1222 1496


# Graphs

In [25]:
true = [0]*max(lstat['veh_range']+1)
ovr = [0]*max(lstat['veh_range']+1)

In [26]:
for i in lstat.index.values:
    if lstat.at[i,'Pred'] == 1:
        true[lstat.at[i,'veh_range']]+=1
    ovr[lstat.at[i,'veh_range']]+=1

In [34]:
for i in mstat.index.values:
    if mstat.at[i,'Pred'] == 1:
        true[mstat.at[i,'veh_range']]+=1
    ovr[mstat.at[i,'veh_range']]+=1

In [35]:
for i in hstat.index.values:
    if hstat.at[i,'Pred'] == 1:
        true[hstat.at[i,'veh_range']]+=1
    ovr[hstat.at[i,'veh_range']]+=1

In [27]:
acc = [i / j for i, j in zip(true, ovr)]

In [31]:
plt.xlabel('Number of surrounding vehicles(120m)')
plt.ylabel('Average accuracy')
plt.bar(range(0,len(acc)-1),acc[:-1])

<BarContainer object of 19 artists>

In [50]:
acc[15]

0.4431818181818182

In [24]:
# lstat.to_pickle(f'/home/mohit/webots_code/data/final/test_stats_new.pkl')